
# Taller de MongoDB con Python

Este notebook contiene un ejercicio guiado para trabajar con MongoDB Atlas y la librería `pymongo` en Python.  
Escenario: un servicio de streaming que gestiona **users, movies y theaters**.

---



## 1. Instalación de dependencias

Ejecuta la celda para instalar `pymongo` (si no lo tienes ya).


In [1]:
!pip install pymongo

In [2]:
!python3 --version

Python 3.10.12



## 2. Conexión a MongoDB Atlas

Copia tu cadena de conexión de MongoDB Atlas en la variable `MONGO_URI`.  
Si no tienes cuenta, puedes crear una gratuita en: https://www.mongodb.com/atlas/database


In [5]:
from pymongo import MongoClient

MONGO_URI = "mongodb://localhost:27017/"

client = MongoClient(MONGO_URI)

# Test de conexion
try:
    client.admin.command("ping")
    print("Exitoso!")
except Exception as e:
    print("Error:", e)

Exitoso!


In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

# Configuración de conexión
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "postgres"
POSTGRES_DB = "db_biblioteca"
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432

SQLALCHEMY_DATABASE_URL = (
    f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)

# Crear motor (engine)
engine = create_engine(SQLALCHEMY_DATABASE_URL)

# Sesión
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Base para modelos
Base = declarative_base()

# Dependencia para obtener sesión
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [10]:
client

MongoClient(host=['ac-3iqvajp-shard-00-00.geguvgm.mongodb.net:27017', 'ac-3iqvajp-shard-00-02.geguvgm.mongodb.net:27017', 'ac-3iqvajp-shard-00-01.geguvgm.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='ClusterData', authsource='admin', replicaset='atlas-8t21oq-shard-0', tls=True)

In [26]:
db = client["sample_mflix"]
movies = db["movies"]
theaters = db["theaters"]
users = db["users"]

#### Encuentra la pelicula "The Great Train Robbery"

In [27]:
for doc in movies.find({"title":'The Great Train Robbery'}):
    print(doc)

OperationFailure: bad auth : authentication failed, full error: {'ok': 0, 'errmsg': 'bad auth : authentication failed', 'code': 8000, 'codeName': 'AtlasError'}

In [ ]:
great = []
for doc in movies.find({"title":'The Great Train Robbery'}):
    great.append(doc)

In [ ]:
great[1]["cast"]


## 3. Insertar datos de ejemplo

 Se usarían los siguientes comandos para insertar comandos en caso de tener colecciones vacías. Esta celda muestra cómo insertar documentos iniciales en las colecciones.


In [ ]:
from datetime import datetime

if usuarios.estimated_document_count() == 0:
    usuarios.insert_many([
        {"_id": 1001, "nombre": "Ana", "pais": "ES", "plan": "Premium"},
        {"_id": 1002, "nombre": "Luis", "pais": "ES", "plan": "Básico"},
        {"_id": 1003, "nombre": "Marta", "pais": "MX", "plan": "Premium"},
    ])
    print("Usuarios insertados.")

if peliculas.estimated_document_count() == 0:
    peliculas.insert_many([
        {"_id": "m1", "titulo": "Inception", "anio": 2010, "genero": "SciFi"},
        {"_id": "m2", "titulo": "Matrix", "anio": 1999, "genero": "SciFi"},
        {"_id": "m3", "titulo": "Amélie", "anio": 2001, "genero": "Comedia"},
    ])
    print("Películas insertadas.")

if reproducciones.estimated_document_count() == 0:
    reproducciones.insert_many([
        {"usuario_id": 1001, "pelicula_id": "m1", "inicio": datetime(2025, 9, 1, 20, 0), "progreso": "00:32:15"},
        {"usuario_id": 1002, "pelicula_id": "m2", "inicio": datetime(2025, 9, 2, 21, 15), "progreso": "00:45:22"},
        {"usuario_id": 1001, "pelicula_id": "m2", "inicio": datetime(2025, 9, 3, 19, 30), "progreso": "01:10:05"},
        {"usuario_id": 1003, "pelicula_id": "m3", "inicio": datetime(2025, 9, 4, 18, 5),  "progreso": "00:10:03"},
        {"usuario_id": 1002, "pelicula_id": "m3", "inicio": datetime(2025, 9, 5, 22, 0),  "progreso": "00:05:11"},
    ])
    print("Reproducciones insertadas.")



## 4. Consultas básicas (find)


In [ ]:

from pprint import pprint

print("Usuarios Premium:")
for doc in usuarios.find({"plan": "Premium"}, {"_id": 1, "nombre": 1, "pais": 1}):
    pprint(doc)

print("\nPelículas de SciFi (título y año):")
for doc in peliculas.find({"genero": "SciFi"}, {"_id": 0, "titulo": 1, "anio": 1}):
    pprint(doc)

print("\nReproducciones del usuario 1001:")
for doc in reproducciones.find({"usuario_id": 1001}):
    pprint(doc)



## 5. Actualizar y eliminar documentos


In [ ]:

print("Actualizar el plan de Luis (1002) a 'Standard'...")
res = usuarios.update_one({"_id": 1002}, {"$set": {"plan": "Standard"}})
print("Matched:", res.matched_count, "Modified:", res.modified_count)
pprint(usuarios.find_one({"_id": 1002}))

print("\nEliminar una reproducción muy corta...")
res = reproducciones.delete_one({"progreso": "00:05:11"})
print("Eliminados:", res.deleted_count)



## 6. Agregación: Top películas más vistas


In [ ]:

pipeline = [
    {"$group": {"_id": "$pelicula_id", "conteo": {"$sum": 1}}},
    {"$sort": {"conteo": -1}},
    {"$limit": 5},
    {"$lookup": {
        "from": "peliculas",
        "localField": "_id",
        "foreignField": "_id",
        "as": "pelicula"
    }},
    {"$unwind": "$pelicula"},
    {"$project": {"_id": 0, "pelicula": "$pelicula.titulo", "veces": "$conteo"}}
]

for doc in reproducciones.aggregate(pipeline):
    pprint(doc)
